In [1]:
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC



import seaborn as sns

In [2]:
pd.set_option('display.max_rows', 300)

In [3]:
survey = pd.read_csv('/Users/williamyee/Documents/Data Bootcamp/Classification-Project-DSjobs/Data/kaggle-survey-2017/multi_r_.csv', encoding = 'ISO-8859-1')

/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (32,84,87,88,100,110,117,124,125,128,130,131,165) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
survey.head()

,Unnamed: 0,GenderSelect,Country,Age,EmploymentStatus,StudentStatus,LearningDataScience,CodeWriter,CareerSwitcher,CurrentJobTitleSelect,...,JobFactorExperienceLevel,JobFactorDepartment,JobFactorTitle,JobFactorCompanyFunding,JobFactorImpact,JobFactorRemote,JobFactorIndustry,JobFactorLeaderReputation,JobFactorDiversity,JobFactorPublishingOpportunity
0,1,"Non-binary, genderqueer, or gender non-conforming",NaN,NaN,Employed full-time,NaN,NaN,Yes,NaN,DBA/Database Engineer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Female,United States,30.0,"Not employed, but looking for work",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somewhat important,NaN,NaN
2,3,Male,Canada,28.0,"Not employed, but looking for work",NaN,NaN,NaN,NaN,NaN,...,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important
3,4,Male,United States,56.0,"Independent contractor, freelancer, or self-em...",NaN,NaN,Yes,NaN,Operations Research Practitioner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Male,Taiwan,38.0,Employed full-time,NaN,NaN,Yes,NaN,Computer Scientist,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
survey.EmploymentStatus.unique()

array(['Employed full-time', 'Not employed, but looking for work',
       'Independent contractor, freelancer, or self-employed',
       'Not employed, and not looking for work', 'Employed part-time',
       'Retired', 'I prefer not to say'], dtype=object)

In [6]:
survey.groupby('EmploymentStatus').count()

,Unnamed: 0,GenderSelect,Country,Age,StudentStatus,LearningDataScience,CodeWriter,CareerSwitcher,CurrentJobTitleSelect,TitleFit,...,JobFactorExperienceLevel,JobFactorDepartment,JobFactorTitle,JobFactorCompanyFunding,JobFactorImpact,JobFactorRemote,JobFactorIndustry,JobFactorLeaderReputation,JobFactorDiversity,JobFactorPublishingOpportunity
EmploymentStatus,,,,,,,,,,,,,,,,,,,,,
Employed full-time,10897,10853,10840,10702,0,0,10843,2400,9787,9533,...,1066,1060,1061,1062,1062,1068,1065,1063,1065,1075
Employed part-time,917,910,907,901,0,0,903,288,787,751,...,132,131,132,131,132,133,132,134,133,132
I prefer not to say,420,404,399,382,380,383,0,0,0,0,...,190,188,189,187,187,188,188,187,187,187
"Independent contractor, freelancer, or self-employed",1330,1321,1321,1302,0,0,1322,281,1176,1142,...,105,104,104,104,104,106,106,103,104,105
"Not employed, and not looking for work",924,919,915,909,900,901,0,0,0,0,...,522,522,519,521,519,525,520,521,524,520
"Not employed, but looking for work",2110,2098,2096,2075,0,0,0,0,0,0,...,1422,1411,1409,1406,1390,1404,1398,1393,1397,1405
Retired,118,116,117,114,0,0,118,43,80,78,...,0,0,0,0,0,0,0,0,0,0


In [7]:
survey.UniversityImportance.value_counts()

Very important                                 2925
Important                                      2347
Somewhat important                             1695
Not very important                              772
Not at all important                            317
N/A, I did not receive any formal education      42
Name: UniversityImportance, dtype: int64

In [49]:
conditions = [  survey.UniversityImportance.isin(['Very important', 'Important', 'Somewhat important']),
                survey.UniversityImportance.isin(['Not very important','Not at all important','N/A, I did not receive any formal education'])
             ]
choices = ['Important' , 'Not Important']
survey['UniversityImportance'] = np.select(conditions, choices, default=survey.UniversityImportance)

In [50]:
survey.UniversityImportance.value_counts()

Important        6967
Not Important    1131
Name: UniversityImportance, dtype: int64

In [51]:
survey.TitleFit.value_counts()

Fine         5197
Perfectly    1615
Poorly       1101
Name: TitleFit, dtype: int64

In [52]:
survey.GenderSelect.value_counts()

Male                                                 6795
Female                                               1193
A different identity                                   56
Non-binary, genderqueer, or gender non-conforming      37
Name: GenderSelect, dtype: int64

In [53]:
survey.EmploymentStatus.value_counts()

Employed full-time                                      6805
Independent contractor, freelancer, or self-employed     847
Employed part-time                                       446
Name: EmploymentStatus, dtype: int64

In [54]:
survey.CareerSwitcher.value_counts()

Series([], Name: CareerSwitcher, dtype: int64)

In [55]:
survey.CurrentEmployerType.value_counts()

Employed by a company that performs advanced analytics                                                                                                                                                                                           1742
Employed by professional services/consulting firm                                                                                                                                                                                                1528
Employed by college or university                                                                                                                                                                                                                1226
Employed by a company that doesn't perform advanced analytics                                                                                                                                                                                    1143
Employed by comp

In [56]:
survey.Tenure.value_counts()

3 to 5 years                          2401
1 to 2 years                          1869
More than 10 years                    1651
6 to 10 years                         1388
Less than a year                       726
I don't write code to analyze data      42
Name: Tenure, dtype: int64

In [57]:
survey.LearningDataScienceTime.value_counts()

Series([], Name: LearningDataScienceTime, dtype: int64)

In [58]:
survey.CurrentJobTitleSelect.value_counts()

Data Scientist                          2105
Software Developer/Software Engineer     931
Data Analyst                             896
Scientist/Researcher                     749
Other                                    669
Machine Learning Engineer                494
Business Analyst                         467
Researcher                               450
Engineer                                 295
Computer Scientist                       230
Statistician                             221
Programmer                               201
Predictive Modeler                       150
DBA/Database Engineer                    119
Data Miner                                78
Operations Research Practitioner          41
Name: CurrentJobTitleSelect, dtype: int64

In [59]:
survey.StudentStatus.value_counts()

Series([], Name: StudentStatus, dtype: int64)

In [60]:
survey = survey[survey.UniversityImportance.notna()]
survey.shape

(8098, 229)

In [61]:
percent_missing = survey.isnull().sum()*100/len(survey)
missing_value_df = pd.DataFrame({'columnname':survey.columns, 'percent_missing':percent_missing})
seventy_five_data = missing_value_df[missing_value_df.percent_missing<25]

In [62]:
seventy_five_data

,columnname,percent_missing
Unnamed: 0,Unnamed: 0,0.000000
GenderSelect,GenderSelect,0.209928
Country,Country,0.321067
Age,Age,1.420104
EmploymentStatus,EmploymentStatus,0.000000
CodeWriter,CodeWriter,0.000000
CurrentJobTitleSelect,CurrentJobTitleSelect,0.024697
TitleFit,TitleFit,2.284515
CurrentEmployerType,CurrentEmployerType,1.457150
MLToolNextYearSelect,MLToolNextYearSelect,20.782909


In [63]:
survey_minimal_missing= survey[list(seventy_five_data.columnname)]
X = survey_minimal_missing.drop(columns = 'UniversityImportance')
y = survey_minimal_missing['UniversityImportance']
X

,Unnamed: 0,GenderSelect,Country,Age,EmploymentStatus,CodeWriter,CurrentJobTitleSelect,TitleFit,CurrentEmployerType,MLToolNextYearSelect,...,WorkChallengesSelect,WorkDataVisualizations,WorkInternalVsExternalTools,WorkMLTeamSeatSelect,WorkDataStorage,WorkDataSharing,WorkCodeSharing,RemoteWork,SalaryChange,JobSatisfaction
0,1,"Non-binary, genderqueer, or gender non-conforming",NaN,NaN,Employed full-time,Yes,DBA/Database Engineer,Fine,Employed by a company that doesn't perform adv...,SAS Base,...,Company politics / Lack of management/financia...,26-50% of projects,Do not know,Standalone Team,Document-oriented (e.g. MongoDB/Elasticsearch)...,"Company Developed Platform,I don't typically s...","Mercurial,Subversion,Other",Always,I am not currently employed,5
3,4,Male,United States,56.0,"Independent contractor, freelancer, or self-em...",Yes,Operations Research Practitioner,Poorly,Self-employed,TensorFlow,...,Company politics / Lack of management/financia...,100% of projects,Entirely internal,Standalone Team,"Column-oriented relational (e.g. KDB/MariaDB),...","Company Developed Platform,Email",Generic cloud file sharing software (Dropbox/B...,NaN,Has increased 20% or more,10 - Highly Satisfied
4,5,Male,Taiwan,38.0,Employed full-time,Yes,Computer Scientist,Fine,Employed by a company that doesn't perform adv...,TensorFlow,...,Company politics / Lack of management/financia...,10-25% of projects,Approximately half internal and half external,Business Department,Flat files not in a database or cache (e.g. CS...,Company Developed Platform,Git,Rarely,I do not want to share information about my sa...,2
5,6,Male,Brazil,46.0,Employed full-time,Yes,Data Scientist,Fine,Employed by college or university,TensorFlow,...,Dirty data,76-99% of projects,More internal than external,Other,"Column-oriented relational (e.g. KDB/MariaDB),...",I don't typically share data,"Bitbucket,Git,Other",Sometimes,I do not want to share information about my sa...,8
6,7,Male,United States,35.0,Employed full-time,Yes,Computer Scientist,Fine,Employed by government,TensorFlow,...,Did not instrument data useful for scientific ...,26-50% of projects,Entirely internal,IT Department,Flat files not in a database or cache (e.g. CS...,"Company Developed Platform,Email","Bitbucket,Git",NaN,NaN,8
8,9,Female,Australia,43.0,Employed full-time,Yes,Business Analyst,Fine,Employed by non-profit or NGO,Microsoft Excel Data Mining,...,Dirty data,Less than 10% of projects,Entirely internal,IT Department,Row-oriented relational (e.g. MySQL/Microsoft ...,"I don't typically share data,Share Drive/Share...",Generic non-cloud file sharing software (Email...,Rarely,Has stayed about the same (has not increased o...,7
9,10,Male,Russia,33.0,Employed full-time,Yes,Software Developer/Software Engineer,Fine,Employed by a company that doesn't perform adv...,C/C++,...,Data Science results not used by business deci...,Less than 10% of projects,Do not know,IT Department,Document-oriented (e.g. MongoDB/Elasticsearch)...,I don't typically share data,"Bitbucket,Git",Rarely,Has increased between 6% and 19%,7
11,12,Male,India,27.0,Employed full-time,Yes,Data Scientist,Fine,Employed by professional services/consulting f...,Other,...,"Explaining data science to others,Lack of data...",26-50% of projects,Entirely internal,Standalone Team,Document-oriented (e.g. MongoDB/Elasticsearch)...,Company Developed Platform,Bitbucket,Rarely,Has increased 20% or more,7
14,15,Male,Taiwan,26.0,Employed full-time,Yes,Software Developer/Software Engineer,Fine,Employed by professional services/consulting firm,TensorFlow,...,Company politics / Lack of management/financia...,26-50% of projects,Do not know,IT Department,Flat files not in a database or cache (e.g. CS...,"Email,Share Drive/SharePoint",Git,Sometimes,Has stayed about the same (has not increased o...,5
15,16,Male,United States,58.0,"Independent contractor, freelancer, or self-em...",Yes,DBA/Database Engineer,Poorly,Employed by professional services/consulting firm,Python,

In [64]:
imp_freq = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_imputed = imp_freq.fit_transform(X)

In [65]:
X_imputed

array([[1, 'Non-binary, genderqueer, or gender non-conforming',
        'United States', ..., 'Always', 'I am not currently employed', '5'],
       [4, 'Male', 'United States', ..., 'Sometimes',
        'Has increased 20% or more', '10 - Highly Satisfied'],
       [5, 'Male', 'Taiwan', ..., 'Rarely',
        'I do not want to share information about my salary/compensation',
        '2'],
       ..., 
       [16703, 'Male', 'Australia', ..., 'Sometimes',
        'Has increased 20% or more', '7'],
       [16707, 'Male', 'Australia', ..., 'Sometimes',
        'Has increased 20% or more', '7'],
       [16713, 'Male', 'Indonesia', ..., 'Rarely',
        'I do not want to share information about my salary/compensation',
        '7']], dtype=object)

In [66]:
enc = OneHotEncoder(handle_unknown= 'ignore')
X_imputed= enc.fit_transform(X_imputed)


In [67]:
X, X_test, y, y_test = train_test_split(X_imputed, y, test_size=.2, random_state=100)


In [83]:
clf = RandomForestClassifier(max_depth=4,random_state=7)
clf.fit(X,y)

/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=7, verbose=0, warm_start=False)

In [84]:
clf.score(X_test, y_test)

0.84999999999999998

In [85]:
list(clf.feature_importances_)

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [70]:
knn = KNeighborsClassifier(n_neighbors=35)
knn.fit(X,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=35, p=2,
           weights='uniform')

In [71]:
list(knn.predict(X_test))

['Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Important',
 'Impo

In [72]:
knn.score(X_test,y_test)

0.84999999999999998

In [73]:
gbm = GradientBoostingClassifier()
gbm.fit(X,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [74]:
gbm.score(X_test,y_test)

0.85864197530864195

In [75]:
adaboost = AdaBoostClassifier()
adaboost.fit(X,y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [76]:
adaboost.score(X_test,y_test)

0.8598765432098765

In [77]:
linearsvc = LinearSVC()
linearsvc.fit(X,y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [78]:
linearsvc.score(X_test,y_test)

0.83765432098765435

In [79]:
svc = SVC(gamma='scale')
svc.fit(X,y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [80]:
svc.score(X_test,y_test)

0.84999999999999998